In [42]:
# Inventory

In [36]:
import random
from datetime import date, timedelta
import pandas as pd

from sqlalchemy import create_engine, text
from sqlalchemy.exc import IntegrityError

In [37]:
# SQLAlchemy connection string (inside Docker network)
DB_USER = "analytics_user"
DB_PASS = "analyticspass123"
DB_HOST = "mysql"          # service name from docker-compose
DB_PORT = "3306"
DB_NAME = "supply_chain_db"

connection_string = f"mysql+pymysql://{DB_USER}:{DB_PASS}@{DB_HOST}:{DB_PORT}/{DB_NAME}"
engine = create_engine(connection_string)

# Quick test: list tables
with engine.connect() as conn:
    tables = pd.read_sql("SHOW TABLES;", conn)
tables

,Tables_in_supply_chain_db
0,inventory
1,orders
2,price_history
3,products
4,sales
5,suppliers
6,warehouses


In [38]:
def random_date(start_date, end_date):
    delta_days = (end_date - start_date).days
    return start_date + timedelta(days=random.randint(0, delta_days))

In [39]:

def generate_inventory_row():
    product_id = random.randint(1, 46)
    warehouse_id = random.randint(1, 18)

    quantity_on_hand = random.randint(10, 999)

    min_reserved = int(quantity_on_hand * 0.10)
    max_reserved = int(quantity_on_hand * 0.90)
    quantity_reserved = random.randint(min_reserved, max_reserved)

    snapshot_date = random_date(
        date(2020, 1, 1),
        date.today()
    )

    return {
        "product_id": product_id,
        "warehouse_id": warehouse_id,
        "quantity_on_hand": quantity_on_hand,
        "quantity_reserved": quantity_reserved,
        "snapshot_date": snapshot_date
    }


In [40]:
insert_stmt = text("""
INSERT INTO inventory (
    product_id,
    warehouse_id,
    quantity_on_hand,
    quantity_reserved,
    snapshot_date
)
VALUES (
    :product_id,
    :warehouse_id,
    :quantity_on_hand,
    :quantity_reserved,
    :snapshot_date
)
""")


In [41]:
rows_to_insert = 900
inserted = 0

with engine.begin() as conn:  # auto-commit / rollback
    for _ in range(rows_to_insert):
        row = generate_inventory_row()
        try:
            conn.execute(insert_stmt, row)
            inserted += 1
        except IntegrityError:
            # Duplicate (product_id, warehouse_id, snapshot_date)
            pass

print(f"Inserted {inserted} rows successfully.")


Inserted 890 rows successfully.
